In [1]:
import pandas as pd
import numpy as np
import missingno
from collections import Counter


from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")


In [2]:
train = pd.read_csv(r"C:\Users\91939\Documents\DA\ML\titanic\train.csv")
test = pd.read_csv(r"C:\Users\91939\Documents\DA\ML\titanic\test.csv")
ss = pd.read_csv(r"C:\Users\91939\Documents\DA\ML\titanic\gender_submission.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
ss.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [6]:
print("Training set shape", train.shape)
print("Testing set shape", test.shape)
print("Submission set shape", ss.shape)

Training set shape (891, 12)
Testing set shape (418, 11)
Submission set shape (418, 2)


In [7]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [8]:
test.isnull().sum().sort_values(ascending=False)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
dtype: int64

In [9]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [10]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [11]:
train['Sex'].value_counts(dropna=False)

male      577
female    314
Name: Sex, dtype: int64

In [12]:
train[['Sex','Survived']].groupby('Sex',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [13]:
train['Pclass'].value_counts(dropna=False)

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [14]:
train[['Pclass','Survived']].groupby('Pclass',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [15]:
train['Embarked'].value_counts(dropna=False)

S      644
C      168
Q       77
NaN      2
Name: Embarked, dtype: int64

In [16]:
train[['Embarked','Survived']].groupby('Embarked',as_index=False).mean().sort_values(by='Survived',ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


In [17]:
def detect_outliers(df,n,features):
    outlier_indices=[]
    for col in features:
        Q1=np.percentile(df[col],25)
        Q3=np.percentile(df[col],75)
        IQR=Q3-Q1
        outlier_step=1.5*IQR
        outlier_list_col=df[(df[col]<Q1-outlier_step)|(df[col]>Q3+outlier_step)].index
        outlier_indices.extend(outlier_list_col)
    outlier_indices=Counter(outlier_indices)
    multiple_outliers=list(key for key, value in outlier_indices.items() if value>n)
    return multiple_outliers

outliers_to_drop= detect_outliers(train,2,['Age','SibSp','Parch','Fare'])
print("We will drop these {} indices:".format(len(outliers_to_drop)), outliers_to_drop)


We will drop these 10 indices: [27, 88, 159, 180, 201, 324, 341, 792, 846, 863]


In [18]:
train.loc[outliers_to_drop,:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.00,C23 C25 C27,S
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.00,C23 C25 C27,S
159,160,0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,CA. 2343,69.55,NaN,S
180,181,0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.55,NaN,S
201,202,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.55,NaN,S
324,325,0,3,"Sage, Mr. George John Jr",male,NaN,8,2,CA. 2343,69.55,NaN,S
341,342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.00,C23 C25 C27,S
792,793,0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,CA. 2343,69.55,NaN,S
846,847,0,3,"Sage, Mr. Douglas Bullen",male,NaN,8,2,CA. 2343,69.55,NaN,S
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.55,NaN,S


In [19]:
print("Before: {}".format(len(train)))
train=train.drop(outliers_to_drop,axis=0).reset_index(drop=True)
print("After: {}".format(len(train)))

Before: 891
After: 881


In [21]:
train['SibSp'].value_counts(dropna=False)

0    608
1    209
2     28
4     18
3     13
5      5
Name: SibSp, dtype: int64

In [22]:
train[['SibSp','Survived']].groupby('SibSp',as_index=False).mean().sort_values(by='Survived', ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
4,4,0.166667
3,3,0.153846
5,5,0.000000


In [23]:
train['Parch'].value_counts(dropna=False)

0    678
1    118
2     70
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [24]:
train[['Parch','Survived']].groupby('Parch',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.542857
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [25]:
train['Age'].isnull().sum()

170

In [28]:
train['Fare'].isnull().sum()

0

In [29]:
train=train.drop(['Ticket','Cabin'],axis=1)
test=test.drop(['Ticket','Cabin'],axis=1)

In [30]:
train.isnull().sum().sort_values(ascending=False)

Age            170
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Fare             0
dtype: int64

In [31]:
mode=train['Embarked'].dropna().mode()[0]
mode

'S'

In [32]:
train['Embarked'].fillna(mode,inplace=True)

In [33]:
test.isnull().sum().sort_values(ascending=False)

Age            86
Fare            1
PassengerId     0
Pclass          0
Name            0
Sex             0
SibSp           0
Parch           0
Embarked        0
dtype: int64

In [34]:
median=test['Fare'].dropna().median()
median

14.4542

In [35]:
test['Fare'].fillna(median,inplace=True)

In [36]:
combine=pd.concat([train,test],axis=0).reset_index(drop=True)
combine.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [37]:
combine.isnull().sum().sort_values(ascending=False)

Survived       418
Age            256
PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Fare             0
Embarked         0
dtype: int64

In [38]:
combine['Sex']=combine['Sex'].map({'male':0,'female':1})

In [39]:
age_nan_indices=list(combine[combine['Age'].isnull()].index)
len(age_nan_indices)

256

In [40]:
for index in age_nan_indices:
    median_age=combine['Age'].median()
    predict_age=combine['Age'][(combine['SibSp']==combine.iloc[index]['SibSp']) & 
                              (combine['Parch']==combine.iloc[index]['Parch']) & 
                              (combine['Pclass']==combine.iloc[index]['Pclass'])].median()
    if np.isnan(predict_age):
        combine['Age'].iloc[index]=median_age
    else:
        combine['Age'].iloc[index]=predict_age

In [41]:
combine['Age'].isnull().sum()

0

In [42]:
combine['Fare']=combine['Fare'].map(lambda x:np.log(x) if x>0 else 0)

In [44]:
combine.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,1.981001,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,4.266662,C
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,2.070022,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,3.972177,S
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,2.085672,S


In [45]:
combine['Title']=[name.split(',')[1].split('.')[0].strip() for name in combine['Name']]
combine[['Name','Title']].head()

,Name,Title
0,"Braund, Mr. Owen Harris",Mr
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mrs
2,"Heikkinen, Miss. Laina",Miss
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Mrs
4,"Allen, Mr. William Henry",Mr


In [46]:
combine['Title'].value_counts(ascending= False)

Mr              753
Miss            255
Mrs             197
Master           60
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: Title, dtype: int64

In [47]:
combine['Title'].nunique()

18

In [48]:
combine['Title']=combine['Title'].replace(['Dr','Rev','Col','Major','Lady','Jonkheer','Don','Capt','the Countess','Sir','Dona'],
                                         'Rare')
combine['Title']=combine['Title'].replace(['Mlle','Ms'],'Miss')
combine['Title']=combine['Title'].replace('Mme','Mrs')

In [49]:
combine[['Title','Survived']].groupby('Title',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Title,Survived
3,Mrs,0.793651
1,Miss,0.711111
0,Master,0.589744
4,Rare,0.347826
2,Mr,0.157895


In [50]:
combine=combine.drop('Name',axis=1)
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0.0,3,0,22.0,1,0,1.981001,S,Mr
1,2,1.0,1,1,38.0,1,0,4.266662,C,Mrs
2,3,1.0,3,1,26.0,0,0,2.070022,S,Miss
3,4,1.0,1,1,35.0,1,0,3.972177,S,Mrs
4,5,0.0,3,0,35.0,0,0,2.085672,S,Mr


In [51]:
combine['Family Size']=combine['SibSp']+combine['Parch']+1
combine[['SibSp','Parch','Family Size']].head(10)

,SibSp,Parch,Family Size
0,1,0,2
1,1,0,2
2,0,0,1
3,1,0,2
4,0,0,1
5,0,0,1
6,0,0,1
7,3,1,5
8,0,2,3
9,1,0,2


In [52]:
combine[['Family Size','Survived']].groupby('Family Size',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Family Size,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.052632
7,8,0.000000
8,11,NaN


In [53]:
combine['IsAlone']=0
combine.loc[combine['Family Size']==1,'IsAlone']=1

In [54]:
combine[['IsAlone','Survived']].groupby('IsAlone',as_index=False).mean().sort_values(by='IsAlone', ascending=False)

,IsAlone,Survived
1,1,0.303538
0,0,0.514535


In [55]:
combine=combine.drop(['SibSp','Parch','Family Size'],axis=1)
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,1,0.0,3,0,22.0,1.981001,S,Mr,0
1,2,1.0,1,1,38.0,4.266662,C,Mrs,0
2,3,1.0,3,1,26.0,2.070022,S,Miss,1
3,4,1.0,1,1,35.0,3.972177,S,Mrs,0
4,5,0.0,3,0,35.0,2.085672,S,Mr,1


In [56]:
combine['AgeBand']=pd.cut(combine['Age'],5)
combine[['AgeBand','Survived']].groupby('AgeBand',as_index=False).mean().sort_values(by='AgeBand', ascending=False)

,AgeBand,Survived
4,"(64.034, 80.0]",0.090909
3,"(48.068, 64.034]",0.434783
2,"(32.102, 48.068]",0.412844
1,"(16.136, 32.102]",0.339662
0,"(0.0902, 16.136]",0.532110


In [57]:
combine.loc[combine['Age']<=16.136,'Age'] = 0
combine.loc[(combine['Age']>16.136) & (combine['Age']<=32.102),'Age']=1
combine.loc[(combine['Age']>32.102) & (combine['Age']<=48.068),'Age']=2
combine.loc[(combine['Age']>48.068) & (combine['Age']<=64.034),'Age']=3
combine.loc[combine['Age']>64.034,'Age']=4

In [58]:
combine=combine.drop('AgeBand',axis=1)

In [59]:
combine[['Age','Pclass']].dtypes

Age       float64
Pclass      int64
dtype: object

In [60]:
combine['Age']=combine['Age'].astype('int')
combine['Age'].dtypes

dtype('int32')

In [61]:
combine['Age*Class']=combine['Age']*combine['Pclass']
combine[['Age','Pclass','Age*Class']].head()


,Age,Pclass,Age*Class
0,1,3,3
1,2,1,2
2,1,3,3
3,2,1,2
4,2,3,6


In [62]:
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,1,0.0,3,0,1,1.981001,S,Mr,0,3
1,2,1.0,1,1,2,4.266662,C,Mrs,0,2
2,3,1.0,3,1,1,2.070022,S,Miss,1,3
3,4,1.0,1,1,2,3.972177,S,Mrs,0,2
4,5,0.0,3,0,2,2.085672,S,Mr,1,6


In [63]:
combine=pd.get_dummies(combine,columns=['Title'])
combine=pd.get_dummies(combine,columns=['Embarked'], prefix='Em')
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1.981001,0,3,0,0,1,0,0,0,0,1
1,2,1.0,1,1,2,4.266662,0,2,0,0,0,1,0,1,0,0
2,3,1.0,3,1,1,2.070022,1,3,0,1,0,0,0,0,0,1
3,4,1.0,1,1,2,3.972177,0,2,0,0,0,1,0,0,0,1
4,5,0.0,3,0,2,2.085672,1,6,0,0,1,0,0,0,0,1


In [64]:
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1.981001,0,3,0,0,1,0,0,0,0,1
1,2,1.0,1,1,2,4.266662,0,2,0,0,0,1,0,1,0,0
2,3,1.0,3,1,1,2.070022,1,3,0,1,0,0,0,0,0,1
3,4,1.0,1,1,2,3.972177,0,2,0,0,0,1,0,0,0,1
4,5,0.0,3,0,2,2.085672,1,6,0,0,1,0,0,0,0,1


In [65]:
combine['FareBand']=pd.cut(combine['Fare'],5)
combine[['FareBand','Survived']].groupby('FareBand',as_index=False).mean().sort_values(by='FareBand', ascending=False)

,FareBand,Survived
4,"(4.991, 6.239]",0.692308
3,"(3.743, 4.991]",0.669118
2,"(2.496, 3.743]",0.431884
1,"(1.248, 2.496]",0.225627
0,"(-0.00624, 1.248]",0.066667


In [66]:
combine.loc[combine['Fare']<=1.248,'Fare'] = 0
combine.loc[(combine['Fare']>1.248) & (combine['Fare']<=2.496),'Fare']=1
combine.loc[(combine['Fare']>2.496) & (combine['Fare']<=3.743),'Fare']=2
combine.loc[(combine['Fare']>3.743) & (combine['Fare']<=4.991),'Fare']=3
combine.loc[combine['Fare']>4.991,'Fare']=4

In [67]:
combine=combine.drop('FareBand',axis=1)

In [68]:

combine['Fare'].dtypes

dtype('float64')

In [69]:
combine['Fare']=combine['Fare'].astype('int')
combine['Fare'].dtypes

dtype('int32')

In [70]:
train=combine[:len(train)]
test=combine[len(train):]

In [71]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1,0,3,0,0,1,0,0,0,0,1
1,2,1.0,1,1,2,3,0,2,0,0,0,1,0,1,0,0
2,3,1.0,3,1,1,1,1,3,0,1,0,0,0,0,0,1
3,4,1.0,1,1,2,3,0,2,0,0,0,1,0,0,0,1
4,5,0.0,3,0,2,1,1,6,0,0,1,0,0,0,0,1


In [72]:
train=train.drop('PassengerId',axis=1)
train.head()

,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,0.0,3,0,1,1,0,3,0,0,1,0,0,0,0,1
1,1.0,1,1,2,3,0,2,0,0,0,1,0,1,0,0
2,1.0,3,1,1,1,1,3,0,1,0,0,0,0,0,1
3,1.0,1,1,2,3,0,2,0,0,0,1,0,0,0,1
4,0.0,3,0,2,1,1,6,0,0,1,0,0,0,0,1


In [73]:
train['Survived']=train['Survived'].astype('int')
train.head()

,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,0,3,0,1,1,0,3,0,0,1,0,0,0,0,1
1,1,1,1,2,3,0,2,0,0,0,1,0,1,0,0
2,1,3,1,1,1,1,3,0,1,0,0,0,0,0,1
3,1,1,1,2,3,0,2,0,0,0,1,0,0,0,1
4,0,3,0,2,1,1,6,0,0,1,0,0,0,0,1


In [74]:
test.head()


,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
881,892,NaN,3,0,2,1,1,6,0,0,1,0,0,0,1,0
882,893,NaN,3,1,2,1,0,6,0,0,0,1,0,0,0,1
883,894,NaN,2,0,3,1,1,6,0,0,1,0,0,0,1,0
884,895,NaN,3,0,1,1,1,3,0,0,1,0,0,0,0,1
885,896,NaN,3,1,1,2,0,3,0,0,0,1,0,0,0,1


In [75]:
test=test.drop('Survived',axis=1)
test.head()

,PassengerId,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
881,892,3,0,2,1,1,6,0,0,1,0,0,0,1,0
882,893,3,1,2,1,0,6,0,0,0,1,0,0,0,1
883,894,2,0,3,1,1,6,0,0,1,0,0,0,1,0
884,895,3,0,1,1,1,3,0,0,1,0,0,0,0,1
885,896,3,1,1,2,0,3,0,0,0,1,0,0,0,1


In [76]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
881,892,3,0,2,1,1,6,0,0,1,0,0,0,1,0
882,893,3,1,2,1,0,6,0,0,0,1,0,0,0,1
883,894,2,0,3,1,1,6,0,0,1,0,0,0,1,0
884,895,3,0,1,1,1,3,0,0,1,0,0,0,0,1
885,896,3,1,1,2,0,3,0,0,0,1,0,0,0,1


In [77]:
X_train=train.drop('Survived',axis=1)
Y_train= train['Survived']
X_test=test.drop('PassengerId',axis=1)
print('X_train: ', X_train.shape)
print('Y_train: ', Y_train.shape)
print('X_test: ', X_test.shape)

X_train:  (881, 14)
Y_train:  (881,)
X_test:  (418, 14)


In [78]:
logreg=LogisticRegression()
logreg.fit(X_train,Y_train)
Y_pred=logreg.predict(X_test)
acc_log=round(logreg.score(X_train,Y_train)*100,2)
acc_log

80.36

In [79]:
svc=SVC()
svc.fit(X_train,Y_train)
Y_pred=svc.predict(X_test)
acc_svc=round(svc.score(X_train,Y_train)*100,2)
acc_svc

81.73

In [80]:
gaussian=GaussianNB()
gaussian.fit(X_train,Y_train)
Y_pred=gaussian.predict(X_test)
acc_gaussian=round(gaussian.score(X_train,Y_train)*100,2)
acc_gaussian

79.57

In [81]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,Y_train)
Y_pred=knn.predict(X_test)
acc_knn=round(knn.score(X_train,Y_train)*100,2)
acc_knn

83.54

In [82]:
per=Perceptron()
per.fit(X_train,Y_train)
Y_pred=per.predict(X_test)
acc_per=round(per.score(X_train,Y_train)*100,2)
acc_per

76.62

In [83]:
linearsvc=LinearSVC()
linearsvc.fit(X_train,Y_train)
Y_pred=linearsvc.predict(X_test)
acc_linearsvc=round(linearsvc.score(X_train,Y_train)*100,2)
acc_linearsvc

80.48

In [84]:
sgd=SGDClassifier()
sgd.fit(X_train,Y_train)
Y_pred=sgd.predict(X_test)
acc_sgd=round(sgd.score(X_train,Y_train)*100,2)
acc_sgd

79.11

In [85]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train,Y_train)
Y_pred=dtc.predict(X_test)
acc_dtc=round(dtc.score(X_train,Y_train)*100,2)
acc_dtc

86.04

In [86]:
rf=RandomForestClassifier(n_estimators=100)
rf.fit(X_train,Y_train)
Y_pred=rf.predict(X_test)
acc_rf=round(rf.score(X_train,Y_train)*100,2)
acc_rf

86.04

In [87]:
cb=CatBoostClassifier()
cb.fit(X_train,Y_train)
Y_pred=cb.predict(X_test)
acc_cb=round(dtc.score(X_train,Y_train)*100,2)
acc_cb

Learning rate set to 0.00976
0:	learn: 0.6862241	total: 134ms	remaining: 2m 14s
1:	learn: 0.6800601	total: 136ms	remaining: 1m 8s
2:	learn: 0.6737396	total: 138ms	remaining: 45.9s
3:	learn: 0.6677584	total: 141ms	remaining: 35s
4:	learn: 0.6615680	total: 143ms	remaining: 28.5s
5:	learn: 0.6562306	total: 145ms	remaining: 24s
6:	learn: 0.6508071	total: 147ms	remaining: 20.9s
7:	learn: 0.6451461	total: 149ms	remaining: 18.5s
8:	learn: 0.6412020	total: 151ms	remaining: 16.6s
9:	learn: 0.6365301	total: 152ms	remaining: 15.1s
10:	learn: 0.6319632	total: 155ms	remaining: 13.9s
11:	learn: 0.6265906	total: 157ms	remaining: 12.9s
12:	learn: 0.6216745	total: 159ms	remaining: 12.1s
13:	learn: 0.6176746	total: 161ms	remaining: 11.3s
14:	learn: 0.6140204	total: 163ms	remaining: 10.7s
15:	learn: 0.6099341	total: 165ms	remaining: 10.2s
16:	learn: 0.6054250	total: 168ms	remaining: 9.72s
17:	learn: 0.6027931	total: 170ms	remaining: 9.26s
18:	learn: 0.5984658	total: 173ms	remaining: 8.93s
19:	learn: 0.59

178:	learn: 0.3977622	total: 577ms	remaining: 2.65s
179:	learn: 0.3974020	total: 581ms	remaining: 2.64s
180:	learn: 0.3971440	total: 582ms	remaining: 2.63s
181:	learn: 0.3970061	total: 585ms	remaining: 2.63s
182:	learn: 0.3966989	total: 587ms	remaining: 2.62s
183:	learn: 0.3962850	total: 589ms	remaining: 2.61s
184:	learn: 0.3960116	total: 591ms	remaining: 2.6s
185:	learn: 0.3957550	total: 594ms	remaining: 2.6s
186:	learn: 0.3955042	total: 595ms	remaining: 2.59s
187:	learn: 0.3953279	total: 598ms	remaining: 2.58s
188:	learn: 0.3952107	total: 600ms	remaining: 2.58s
189:	learn: 0.3950664	total: 602ms	remaining: 2.57s
190:	learn: 0.3946888	total: 604ms	remaining: 2.56s
191:	learn: 0.3943174	total: 607ms	remaining: 2.56s
192:	learn: 0.3940994	total: 610ms	remaining: 2.55s
193:	learn: 0.3937656	total: 613ms	remaining: 2.54s
194:	learn: 0.3934562	total: 616ms	remaining: 2.54s
195:	learn: 0.3933835	total: 618ms	remaining: 2.54s
196:	learn: 0.3931884	total: 620ms	remaining: 2.53s
197:	learn: 0.

340:	learn: 0.3712637	total: 958ms	remaining: 1.85s
341:	learn: 0.3711225	total: 961ms	remaining: 1.85s
342:	learn: 0.3710142	total: 964ms	remaining: 1.85s
343:	learn: 0.3707850	total: 966ms	remaining: 1.84s
344:	learn: 0.3707336	total: 968ms	remaining: 1.84s
345:	learn: 0.3705978	total: 969ms	remaining: 1.83s
346:	learn: 0.3705882	total: 973ms	remaining: 1.83s
347:	learn: 0.3704690	total: 976ms	remaining: 1.83s
348:	learn: 0.3703382	total: 979ms	remaining: 1.83s
349:	learn: 0.3702459	total: 982ms	remaining: 1.82s
350:	learn: 0.3701035	total: 985ms	remaining: 1.82s
351:	learn: 0.3700305	total: 989ms	remaining: 1.82s
352:	learn: 0.3699518	total: 992ms	remaining: 1.82s
353:	learn: 0.3698385	total: 994ms	remaining: 1.81s
354:	learn: 0.3697101	total: 996ms	remaining: 1.81s
355:	learn: 0.3696154	total: 1000ms	remaining: 1.81s
356:	learn: 0.3695242	total: 1s	remaining: 1.8s
357:	learn: 0.3694127	total: 1s	remaining: 1.8s
358:	learn: 0.3693514	total: 1s	remaining: 1.8s
359:	learn: 0.3692590	t

514:	learn: 0.3565480	total: 1.34s	remaining: 1.26s
515:	learn: 0.3565195	total: 1.34s	remaining: 1.26s
516:	learn: 0.3564437	total: 1.35s	remaining: 1.26s
517:	learn: 0.3563636	total: 1.35s	remaining: 1.25s
518:	learn: 0.3563066	total: 1.35s	remaining: 1.25s
519:	learn: 0.3562565	total: 1.35s	remaining: 1.25s
520:	learn: 0.3561998	total: 1.35s	remaining: 1.25s
521:	learn: 0.3561375	total: 1.36s	remaining: 1.24s
522:	learn: 0.3560397	total: 1.36s	remaining: 1.24s
523:	learn: 0.3559816	total: 1.36s	remaining: 1.24s
524:	learn: 0.3559254	total: 1.36s	remaining: 1.23s
525:	learn: 0.3558783	total: 1.37s	remaining: 1.23s
526:	learn: 0.3557998	total: 1.37s	remaining: 1.23s
527:	learn: 0.3557515	total: 1.37s	remaining: 1.23s
528:	learn: 0.3557437	total: 1.37s	remaining: 1.22s
529:	learn: 0.3557334	total: 1.38s	remaining: 1.22s
530:	learn: 0.3556818	total: 1.38s	remaining: 1.22s
531:	learn: 0.3556746	total: 1.38s	remaining: 1.21s
532:	learn: 0.3555984	total: 1.38s	remaining: 1.21s
533:	learn: 

677:	learn: 0.3481335	total: 1.74s	remaining: 827ms
678:	learn: 0.3480876	total: 1.74s	remaining: 824ms
679:	learn: 0.3480260	total: 1.75s	remaining: 821ms
680:	learn: 0.3480147	total: 1.75s	remaining: 819ms
681:	learn: 0.3479685	total: 1.75s	remaining: 816ms
682:	learn: 0.3479175	total: 1.75s	remaining: 813ms
683:	learn: 0.3478651	total: 1.75s	remaining: 811ms
684:	learn: 0.3478643	total: 1.76s	remaining: 808ms
685:	learn: 0.3477663	total: 1.76s	remaining: 805ms
686:	learn: 0.3477171	total: 1.76s	remaining: 803ms
687:	learn: 0.3476716	total: 1.76s	remaining: 800ms
688:	learn: 0.3476168	total: 1.77s	remaining: 797ms
689:	learn: 0.3475687	total: 1.77s	remaining: 795ms
690:	learn: 0.3475357	total: 1.77s	remaining: 792ms
691:	learn: 0.3474626	total: 1.77s	remaining: 790ms
692:	learn: 0.3474441	total: 1.78s	remaining: 787ms
693:	learn: 0.3473680	total: 1.78s	remaining: 785ms
694:	learn: 0.3473471	total: 1.78s	remaining: 782ms
695:	learn: 0.3472633	total: 1.78s	remaining: 779ms
696:	learn: 

911:	learn: 0.3367457	total: 2.32s	remaining: 224ms
912:	learn: 0.3366977	total: 2.32s	remaining: 221ms
913:	learn: 0.3366486	total: 2.32s	remaining: 218ms
914:	learn: 0.3366119	total: 2.32s	remaining: 216ms
915:	learn: 0.3365710	total: 2.33s	remaining: 213ms
916:	learn: 0.3365197	total: 2.33s	remaining: 211ms
917:	learn: 0.3364398	total: 2.33s	remaining: 208ms
918:	learn: 0.3363744	total: 2.33s	remaining: 206ms
919:	learn: 0.3362882	total: 2.34s	remaining: 203ms
920:	learn: 0.3362337	total: 2.34s	remaining: 201ms
921:	learn: 0.3362153	total: 2.34s	remaining: 198ms
922:	learn: 0.3361668	total: 2.34s	remaining: 196ms
923:	learn: 0.3361305	total: 2.35s	remaining: 193ms
924:	learn: 0.3360640	total: 2.35s	remaining: 191ms
925:	learn: 0.3360393	total: 2.35s	remaining: 188ms
926:	learn: 0.3359870	total: 2.35s	remaining: 186ms
927:	learn: 0.3359369	total: 2.36s	remaining: 183ms
928:	learn: 0.3358963	total: 2.36s	remaining: 180ms
929:	learn: 0.3358685	total: 2.36s	remaining: 178ms
930:	learn: 

86.04

In [88]:
models=pd.DataFrame({'Model':['Log Reg','Super Vector Machines','Gaussian Naive Bayes','KNN','Perceptron','Linear SVC',
                              'Stochastic Gradient descent','Decision Tree','Random Forest','CatBoost'],
                     'Score':[acc_log,acc_svc,acc_gaussian,acc_knn,acc_per,acc_linearsvc,acc_sgd,acc_dtc,acc_rf,acc_cb]})
models.sort_values(by='Score',ascending=False,ignore_index=True)

,Model,Score
0,Decision Tree,86.04
1,Random Forest,86.04
2,CatBoost,86.04
3,KNN,83.54
4,Super Vector Machines,81.73
5,Linear SVC,80.48
6,Log Reg,80.36
7,Gaussian Naive Bayes,79.57
8,Stochastic Gradient descent,79.11
9,Perceptron,76.62


In [89]:
classifiers=[]
classifiers.append(LogisticRegression())
classifiers.append(SVC())
classifiers.append(GaussianNB())
classifiers.append(KNeighborsClassifier(n_neighbors=5))
classifiers.append(Perceptron())
classifiers.append(LinearSVC())
classifiers.append(SGDClassifier())
classifiers.append(DecisionTreeClassifier())
classifiers.append(RandomForestClassifier(n_estimators=100))
classifiers.append(CatBoostClassifier())
len(classifiers)


10

In [90]:
cv_results=[]
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier,X_train,Y_train,scoring='accuracy',cv=10))

Learning rate set to 0.009326
0:	learn: 0.6865909	total: 3.36ms	remaining: 3.36s
1:	learn: 0.6806446	total: 6.32ms	remaining: 3.15s
2:	learn: 0.6750152	total: 9.24ms	remaining: 3.07s
3:	learn: 0.6691525	total: 11.3ms	remaining: 2.81s
4:	learn: 0.6631288	total: 13.5ms	remaining: 2.69s
5:	learn: 0.6579961	total: 16.4ms	remaining: 2.72s
6:	learn: 0.6528587	total: 18.8ms	remaining: 2.66s
7:	learn: 0.6474434	total: 21.5ms	remaining: 2.67s
8:	learn: 0.6435817	total: 23.9ms	remaining: 2.63s
9:	learn: 0.6389850	total: 27.3ms	remaining: 2.7s
10:	learn: 0.6344522	total: 30.5ms	remaining: 2.74s
11:	learn: 0.6302241	total: 34.4ms	remaining: 2.83s
12:	learn: 0.6254402	total: 37.2ms	remaining: 2.83s
13:	learn: 0.6204569	total: 40.1ms	remaining: 2.82s
14:	learn: 0.6157905	total: 41.8ms	remaining: 2.75s
15:	learn: 0.6131331	total: 44.1ms	remaining: 2.71s
16:	learn: 0.6089321	total: 46.3ms	remaining: 2.68s
17:	learn: 0.6044886	total: 49.6ms	remaining: 2.71s
18:	learn: 0.5999208	total: 53.1ms	remaining:

159:	learn: 0.4057820	total: 392ms	remaining: 2.06s
160:	learn: 0.4055510	total: 395ms	remaining: 2.06s
161:	learn: 0.4051522	total: 397ms	remaining: 2.05s
162:	learn: 0.4047357	total: 400ms	remaining: 2.05s
163:	learn: 0.4044643	total: 402ms	remaining: 2.05s
164:	learn: 0.4040364	total: 404ms	remaining: 2.05s
165:	learn: 0.4035545	total: 407ms	remaining: 2.04s
166:	learn: 0.4031554	total: 409ms	remaining: 2.04s
167:	learn: 0.4028039	total: 412ms	remaining: 2.04s
168:	learn: 0.4022936	total: 414ms	remaining: 2.04s
169:	learn: 0.4020358	total: 416ms	remaining: 2.03s
170:	learn: 0.4017044	total: 418ms	remaining: 2.03s
171:	learn: 0.4014668	total: 421ms	remaining: 2.02s
172:	learn: 0.4010322	total: 423ms	remaining: 2.02s
173:	learn: 0.4008025	total: 425ms	remaining: 2.02s
174:	learn: 0.4005310	total: 428ms	remaining: 2.02s
175:	learn: 0.4001286	total: 431ms	remaining: 2.02s
176:	learn: 0.3998257	total: 433ms	remaining: 2.01s
177:	learn: 0.3996335	total: 436ms	remaining: 2.01s
178:	learn: 

327:	learn: 0.3719581	total: 792ms	remaining: 1.62s
328:	learn: 0.3717979	total: 794ms	remaining: 1.62s
329:	learn: 0.3717254	total: 797ms	remaining: 1.62s
330:	learn: 0.3714555	total: 799ms	remaining: 1.61s
331:	learn: 0.3712398	total: 801ms	remaining: 1.61s
332:	learn: 0.3710382	total: 809ms	remaining: 1.62s
333:	learn: 0.3709482	total: 811ms	remaining: 1.62s
334:	learn: 0.3709040	total: 813ms	remaining: 1.61s
335:	learn: 0.3707831	total: 815ms	remaining: 1.61s
336:	learn: 0.3706344	total: 818ms	remaining: 1.61s
337:	learn: 0.3705797	total: 821ms	remaining: 1.61s
338:	learn: 0.3705415	total: 823ms	remaining: 1.6s
339:	learn: 0.3704658	total: 825ms	remaining: 1.6s
340:	learn: 0.3703672	total: 827ms	remaining: 1.6s
341:	learn: 0.3703380	total: 829ms	remaining: 1.59s
342:	learn: 0.3701582	total: 832ms	remaining: 1.59s
343:	learn: 0.3700458	total: 834ms	remaining: 1.59s
344:	learn: 0.3699425	total: 836ms	remaining: 1.59s
345:	learn: 0.3698535	total: 838ms	remaining: 1.58s
346:	learn: 0.3

495:	learn: 0.3575009	total: 1.18s	remaining: 1.2s
496:	learn: 0.3573911	total: 1.19s	remaining: 1.2s
497:	learn: 0.3573569	total: 1.19s	remaining: 1.2s
498:	learn: 0.3572738	total: 1.19s	remaining: 1.2s
499:	learn: 0.3571458	total: 1.19s	remaining: 1.19s
500:	learn: 0.3570618	total: 1.2s	remaining: 1.19s
501:	learn: 0.3570006	total: 1.2s	remaining: 1.19s
502:	learn: 0.3568843	total: 1.2s	remaining: 1.19s
503:	learn: 0.3568278	total: 1.2s	remaining: 1.18s
504:	learn: 0.3567600	total: 1.21s	remaining: 1.18s
505:	learn: 0.3567000	total: 1.21s	remaining: 1.18s
506:	learn: 0.3566335	total: 1.21s	remaining: 1.18s
507:	learn: 0.3565962	total: 1.21s	remaining: 1.17s
508:	learn: 0.3565726	total: 1.21s	remaining: 1.17s
509:	learn: 0.3564763	total: 1.22s	remaining: 1.17s
510:	learn: 0.3564269	total: 1.22s	remaining: 1.17s
511:	learn: 0.3563274	total: 1.22s	remaining: 1.16s
512:	learn: 0.3562644	total: 1.22s	remaining: 1.16s
513:	learn: 0.3562335	total: 1.22s	remaining: 1.16s
514:	learn: 0.356222

680:	learn: 0.3464263	total: 1.57s	remaining: 737ms
681:	learn: 0.3464129	total: 1.58s	remaining: 735ms
682:	learn: 0.3463263	total: 1.58s	remaining: 733ms
683:	learn: 0.3463073	total: 1.58s	remaining: 730ms
684:	learn: 0.3462621	total: 1.58s	remaining: 728ms
685:	learn: 0.3462284	total: 1.58s	remaining: 725ms
686:	learn: 0.3461968	total: 1.59s	remaining: 723ms
687:	learn: 0.3461932	total: 1.59s	remaining: 721ms
688:	learn: 0.3461619	total: 1.59s	remaining: 719ms
689:	learn: 0.3460750	total: 1.59s	remaining: 716ms
690:	learn: 0.3460571	total: 1.6s	remaining: 714ms
691:	learn: 0.3460095	total: 1.6s	remaining: 711ms
692:	learn: 0.3459390	total: 1.6s	remaining: 709ms
693:	learn: 0.3459146	total: 1.6s	remaining: 706ms
694:	learn: 0.3457758	total: 1.6s	remaining: 704ms
695:	learn: 0.3456979	total: 1.61s	remaining: 702ms
696:	learn: 0.3456227	total: 1.61s	remaining: 700ms
697:	learn: 0.3455369	total: 1.61s	remaining: 697ms
698:	learn: 0.3454352	total: 1.61s	remaining: 695ms
699:	learn: 0.345

...

309:	learn: 0.3777298	total: 827ms	remaining: 1.84s
310:	learn: 0.3776065	total: 829ms	remaining: 1.84s
311:	learn: 0.3774451	total: 832ms	remaining: 1.83s
312:	learn: 0.3772776	total: 835ms	remaining: 1.83s
313:	learn: 0.3772298	total: 837ms	remaining: 1.83s
314:	learn: 0.3771532	total: 840ms	remaining: 1.83s
315:	learn: 0.3770397	total: 842ms	remaining: 1.82s
316:	learn: 0.3770284	total: 844ms	remaining: 1.82s
317:	learn: 0.3769496	total: 846ms	remaining: 1.81s
318:	learn: 0.3768337	total: 849ms	remaining: 1.81s
319:	learn: 0.3767254	total: 850ms	remaining: 1.81s
320:	learn: 0.3766475	total: 852ms	remaining: 1.8s
321:	learn: 0.3763959	total: 855ms	remaining: 1.8s
322:	learn: 0.3763413	total: 856ms	remaining: 1.79s
323:	learn: 0.3763377	total: 858ms	remaining: 1.79s
324:	learn: 0.3763223	total: 860ms	remaining: 1.79s
325:	learn: 0.3762021	total: 862ms	remaining: 1.78s
326:	learn: 0.3761730	total: 864ms	remaining: 1.78s
327:	learn: 0.3761662	total: 865ms	remaining: 1.77s
328:	learn: 0.

476:	learn: 0.3634596	total: 1.21s	remaining: 1.33s
477:	learn: 0.3633725	total: 1.22s	remaining: 1.33s
478:	learn: 0.3633243	total: 1.22s	remaining: 1.32s
479:	learn: 0.3632741	total: 1.22s	remaining: 1.32s
480:	learn: 0.3631770	total: 1.22s	remaining: 1.32s
481:	learn: 0.3631214	total: 1.23s	remaining: 1.32s
482:	learn: 0.3630524	total: 1.23s	remaining: 1.31s
483:	learn: 0.3630086	total: 1.23s	remaining: 1.31s
484:	learn: 0.3630048	total: 1.23s	remaining: 1.31s
485:	learn: 0.3628569	total: 1.23s	remaining: 1.3s
486:	learn: 0.3628275	total: 1.24s	remaining: 1.3s
487:	learn: 0.3627331	total: 1.24s	remaining: 1.3s
488:	learn: 0.3625657	total: 1.24s	remaining: 1.3s
489:	learn: 0.3624713	total: 1.24s	remaining: 1.29s
490:	learn: 0.3624191	total: 1.25s	remaining: 1.29s
491:	learn: 0.3623490	total: 1.25s	remaining: 1.29s
492:	learn: 0.3622919	total: 1.25s	remaining: 1.29s
493:	learn: 0.3622588	total: 1.25s	remaining: 1.28s
494:	learn: 0.3621381	total: 1.25s	remaining: 1.28s
495:	learn: 0.36

646:	learn: 0.3521814	total: 1.6s	remaining: 873ms
647:	learn: 0.3521776	total: 1.6s	remaining: 871ms
648:	learn: 0.3521694	total: 1.6s	remaining: 868ms
649:	learn: 0.3521360	total: 1.61s	remaining: 866ms
650:	learn: 0.3521150	total: 1.61s	remaining: 863ms
651:	learn: 0.3521016	total: 1.61s	remaining: 860ms
652:	learn: 0.3520177	total: 1.61s	remaining: 858ms
653:	learn: 0.3519523	total: 1.62s	remaining: 856ms
654:	learn: 0.3518466	total: 1.62s	remaining: 853ms
655:	learn: 0.3517861	total: 1.62s	remaining: 851ms
656:	learn: 0.3517783	total: 1.62s	remaining: 848ms
657:	learn: 0.3517302	total: 1.63s	remaining: 846ms
658:	learn: 0.3516831	total: 1.63s	remaining: 843ms
659:	learn: 0.3516262	total: 1.63s	remaining: 840ms
660:	learn: 0.3515832	total: 1.63s	remaining: 837ms
661:	learn: 0.3515481	total: 1.64s	remaining: 835ms
662:	learn: 0.3514998	total: 1.64s	remaining: 832ms
663:	learn: 0.3514355	total: 1.64s	remaining: 829ms
664:	learn: 0.3513895	total: 1.64s	remaining: 827ms
665:	learn: 0.3

814:	learn: 0.3435210	total: 1.99s	remaining: 451ms
815:	learn: 0.3434884	total: 1.99s	remaining: 449ms
816:	learn: 0.3434101	total: 1.99s	remaining: 446ms
817:	learn: 0.3433695	total: 1.99s	remaining: 444ms
818:	learn: 0.3433127	total: 2s	remaining: 441ms
819:	learn: 0.3432505	total: 2s	remaining: 439ms
820:	learn: 0.3431882	total: 2s	remaining: 436ms
821:	learn: 0.3431618	total: 2s	remaining: 434ms
822:	learn: 0.3431323	total: 2.01s	remaining: 432ms
823:	learn: 0.3430718	total: 2.01s	remaining: 429ms
824:	learn: 0.3430242	total: 2.01s	remaining: 427ms
825:	learn: 0.3429996	total: 2.01s	remaining: 424ms
826:	learn: 0.3429440	total: 2.02s	remaining: 422ms
827:	learn: 0.3428910	total: 2.02s	remaining: 419ms
828:	learn: 0.3428046	total: 2.02s	remaining: 416ms
829:	learn: 0.3427572	total: 2.02s	remaining: 414ms
830:	learn: 0.3427015	total: 2.02s	remaining: 412ms
831:	learn: 0.3426559	total: 2.03s	remaining: 409ms
832:	learn: 0.3426177	total: 2.03s	remaining: 407ms
833:	learn: 0.3424773	to

986:	learn: 0.3352908	total: 2.38s	remaining: 31.4ms
987:	learn: 0.3352343	total: 2.38s	remaining: 29ms
988:	learn: 0.3351719	total: 2.39s	remaining: 26.5ms
989:	learn: 0.3351221	total: 2.39s	remaining: 24.1ms
990:	learn: 0.3350773	total: 2.39s	remaining: 21.7ms
991:	learn: 0.3350426	total: 2.39s	remaining: 19.3ms
992:	learn: 0.3349820	total: 2.4s	remaining: 16.9ms
993:	learn: 0.3349439	total: 2.4s	remaining: 14.5ms
994:	learn: 0.3348946	total: 2.4s	remaining: 12.1ms
995:	learn: 0.3348262	total: 2.4s	remaining: 9.65ms
996:	learn: 0.3347895	total: 2.41s	remaining: 7.24ms
997:	learn: 0.3347488	total: 2.41s	remaining: 4.83ms
998:	learn: 0.3347019	total: 2.41s	remaining: 2.41ms
999:	learn: 0.3346525	total: 2.41s	remaining: 0us


In [91]:
cv_mean=[]
cv_std=[]
for cv_result in cv_results:
    cv_mean.append(cv_result.mean())
    cv_std.append(cv_result.std())

In [92]:
cv_res=pd.DataFrame({'Cross Validation Mean':cv_mean,'Cross Validation Std':cv_std,'Algorithm':['Log Reg',
                            'Super Vector Machines','Gaussian Naive Bayes','KNN','Perceptron','Linear SVC',
                              'Stochastic Gradient descent','Decision Tree','Random Forest','CatBoost']})
cv_res.sort_values(by='Cross Validation Mean', ascending=False,ignore_index=True)

,Cross Validation Mean,Cross Validation Std,Algorithm
0,0.804763,0.033268,CatBoost
1,0.801379,0.024871,KNN
2,0.801366,0.036706,Log Reg
3,0.799119,0.051465,Super Vector Machines
4,0.799106,0.031242,Decision Tree
5,0.794535,0.039259,Random Forest
6,0.793424,0.035446,Linear SVC
7,0.787768,0.042377,Gaussian Naive Bayes
8,0.773085,0.063378,Stochastic Gradient descent
9,0.745761,0.067764,Perceptron


In [93]:
param_grid={'max_depth': [3,4,5],'n_estimators':[100, 200, 300]}
grid=GridSearchCV(CatBoostClassifier(),param_grid,refit=True,verbose=3)
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Learning rate set to 0.073257
0:	learn: 0.6657642	total: 854us	remaining: 84.6ms
1:	learn: 0.6432617	total: 1.65ms	remaining: 80.7ms
2:	learn: 0.6218895	total: 2.43ms	remaining: 78.5ms
3:	learn: 0.6053936	total: 3.18ms	remaining: 76.4ms
4:	learn: 0.5883245	total: 3.85ms	remaining: 73.1ms
5:	learn: 0.5726233	total: 4.38ms	remaining: 68.7ms
6:	learn: 0.5582588	total: 5.15ms	remaining: 68.4ms
7:	learn: 0.5462284	total: 5.98ms	remaining: 68.7ms
8:	learn: 0.5367629	total: 6.65ms	remaining: 67.2ms
9:	learn: 0.5261393	total: 7.44ms	remaining: 66.9ms
10:	learn: 0.5173289	total: 8.42ms	remaining: 68.2ms
11:	learn: 0.5082165	total: 9.62ms	remaining: 70.5ms
12:	learn: 0.5010786	total: 10.5ms	remaining: 70.6ms
13:	learn: 0.4936761	total: 11.3ms	remaining: 69.2ms
14:	learn: 0.4880863	total: 12ms	remaining: 68.1ms
15:	learn: 0.4830754	total: 12.7ms	remaining: 66.7ms
16:	learn: 0.4788554	total: 13.3ms	remaining: 65ms
17:	learn: 0.4749230	tot

Learning rate set to 0.073301
0:	learn: 0.6651516	total: 793us	remaining: 78.6ms
1:	learn: 0.6427076	total: 1.47ms	remaining: 72ms
2:	learn: 0.6217484	total: 2.11ms	remaining: 68.3ms
3:	learn: 0.6044986	total: 2.8ms	remaining: 67.2ms
4:	learn: 0.5880154	total: 3.31ms	remaining: 62.8ms
5:	learn: 0.5729336	total: 3.9ms	remaining: 61.1ms
6:	learn: 0.5599447	total: 4.58ms	remaining: 60.9ms
7:	learn: 0.5485505	total: 5.17ms	remaining: 59.5ms
8:	learn: 0.5397277	total: 5.75ms	remaining: 58.2ms
9:	learn: 0.5296676	total: 6.5ms	remaining: 58.5ms
10:	learn: 0.5216338	total: 7.22ms	remaining: 58.4ms
11:	learn: 0.5139929	total: 7.9ms	remaining: 57.9ms
12:	learn: 0.5071515	total: 8.65ms	remaining: 57.9ms
13:	learn: 0.5002953	total: 9.18ms	remaining: 56.4ms
14:	learn: 0.4947435	total: 9.89ms	remaining: 56ms
15:	learn: 0.4897699	total: 10.6ms	remaining: 55.5ms
16:	learn: 0.4862052	total: 11.2ms	remaining: 54.5ms
17:	learn: 0.4830975	total: 11.7ms	remaining: 53.3ms
18:	learn: 0.4783333	total: 12.4ms	

Learning rate set to 0.073301
0:	learn: 0.6652239	total: 1.54ms	remaining: 152ms
1:	learn: 0.6438924	total: 2.44ms	remaining: 119ms
2:	learn: 0.6224604	total: 3.06ms	remaining: 99ms
3:	learn: 0.6061327	total: 3.66ms	remaining: 87.8ms
4:	learn: 0.5894354	total: 4.27ms	remaining: 81.1ms
5:	learn: 0.5745834	total: 4.84ms	remaining: 75.9ms
6:	learn: 0.5608860	total: 5.41ms	remaining: 71.9ms
7:	learn: 0.5498024	total: 5.98ms	remaining: 68.8ms
8:	learn: 0.5411225	total: 6.53ms	remaining: 66.1ms
9:	learn: 0.5315612	total: 7.1ms	remaining: 63.9ms
10:	learn: 0.5235027	total: 7.95ms	remaining: 64.3ms
11:	learn: 0.5151348	total: 8.49ms	remaining: 62.2ms
12:	learn: 0.5088303	total: 8.98ms	remaining: 60.1ms
13:	learn: 0.5020138	total: 9.45ms	remaining: 58.1ms
14:	learn: 0.4969373	total: 10.1ms	remaining: 57.2ms
15:	learn: 0.4922753	total: 10.7ms	remaining: 56.4ms
16:	learn: 0.4890923	total: 11.3ms	remaining: 55.1ms
17:	learn: 0.4859886	total: 11.9ms	remaining: 54.2ms
18:	learn: 0.4814128	total: 12.

122:	learn: 0.3953410	total: 228ms	remaining: 143ms
123:	learn: 0.3952090	total: 230ms	remaining: 141ms
124:	learn: 0.3951506	total: 232ms	remaining: 139ms
125:	learn: 0.3949940	total: 234ms	remaining: 137ms
126:	learn: 0.3948608	total: 236ms	remaining: 135ms
127:	learn: 0.3947510	total: 237ms	remaining: 134ms
128:	learn: 0.3945850	total: 240ms	remaining: 132ms
129:	learn: 0.3943446	total: 242ms	remaining: 130ms
130:	learn: 0.3941760	total: 243ms	remaining: 128ms
131:	learn: 0.3940216	total: 245ms	remaining: 126ms
132:	learn: 0.3938768	total: 247ms	remaining: 124ms
133:	learn: 0.3936162	total: 249ms	remaining: 122ms
134:	learn: 0.3934195	total: 251ms	remaining: 121ms
135:	learn: 0.3932123	total: 252ms	remaining: 118ms
136:	learn: 0.3930348	total: 254ms	remaining: 117ms
137:	learn: 0.3926796	total: 256ms	remaining: 115ms
138:	learn: 0.3925063	total: 257ms	remaining: 113ms
139:	learn: 0.3922786	total: 260ms	remaining: 111ms
140:	learn: 0.3920252	total: 262ms	remaining: 109ms
141:	learn: 

96:	learn: 0.3924306	total: 190ms	remaining: 202ms
97:	learn: 0.3915573	total: 193ms	remaining: 201ms
98:	learn: 0.3910066	total: 194ms	remaining: 198ms
99:	learn: 0.3908378	total: 196ms	remaining: 196ms
100:	learn: 0.3905983	total: 198ms	remaining: 194ms
101:	learn: 0.3901295	total: 200ms	remaining: 192ms
102:	learn: 0.3900685	total: 202ms	remaining: 190ms
103:	learn: 0.3896542	total: 204ms	remaining: 189ms
104:	learn: 0.3893657	total: 207ms	remaining: 187ms
105:	learn: 0.3890430	total: 208ms	remaining: 185ms
106:	learn: 0.3887611	total: 210ms	remaining: 182ms
107:	learn: 0.3883606	total: 212ms	remaining: 181ms
108:	learn: 0.3882017	total: 215ms	remaining: 179ms
109:	learn: 0.3878207	total: 218ms	remaining: 179ms
110:	learn: 0.3874874	total: 220ms	remaining: 177ms
111:	learn: 0.3873058	total: 223ms	remaining: 175ms
112:	learn: 0.3870100	total: 225ms	remaining: 173ms
113:	learn: 0.3868716	total: 227ms	remaining: 171ms
114:	learn: 0.3866353	total: 228ms	remaining: 169ms
115:	learn: 0.38

...

111:	learn: 0.3810063	total: 238ms	remaining: 399ms
112:	learn: 0.3806418	total: 241ms	remaining: 399ms
113:	learn: 0.3802513	total: 244ms	remaining: 397ms
114:	learn: 0.3798068	total: 246ms	remaining: 396ms
115:	learn: 0.3792919	total: 248ms	remaining: 393ms
116:	learn: 0.3790844	total: 250ms	remaining: 391ms
117:	learn: 0.3787741	total: 253ms	remaining: 390ms
118:	learn: 0.3785229	total: 255ms	remaining: 388ms
119:	learn: 0.3780214	total: 258ms	remaining: 386ms
120:	learn: 0.3778781	total: 260ms	remaining: 384ms
121:	learn: 0.3775758	total: 262ms	remaining: 383ms
122:	learn: 0.3770976	total: 264ms	remaining: 380ms
123:	learn: 0.3767213	total: 266ms	remaining: 378ms
124:	learn: 0.3764731	total: 268ms	remaining: 376ms
125:	learn: 0.3763816	total: 270ms	remaining: 373ms
126:	learn: 0.3760310	total: 273ms	remaining: 371ms
127:	learn: 0.3756739	total: 275ms	remaining: 369ms
128:	learn: 0.3754326	total: 278ms	remaining: 369ms
129:	learn: 0.3750362	total: 281ms	remaining: 367ms
130:	learn: 

281:	learn: 0.3454234	total: 607ms	remaining: 38.8ms
282:	learn: 0.3453307	total: 610ms	remaining: 36.6ms
283:	learn: 0.3451427	total: 612ms	remaining: 34.5ms
284:	learn: 0.3450087	total: 614ms	remaining: 32.3ms
285:	learn: 0.3449337	total: 616ms	remaining: 30.2ms
286:	learn: 0.3447651	total: 618ms	remaining: 28ms
287:	learn: 0.3446227	total: 621ms	remaining: 25.9ms
288:	learn: 0.3444739	total: 623ms	remaining: 23.7ms
289:	learn: 0.3443595	total: 625ms	remaining: 21.6ms
290:	learn: 0.3442428	total: 627ms	remaining: 19.4ms
291:	learn: 0.3439533	total: 630ms	remaining: 17.2ms
292:	learn: 0.3438131	total: 632ms	remaining: 15.1ms
293:	learn: 0.3437050	total: 634ms	remaining: 12.9ms
294:	learn: 0.3435117	total: 636ms	remaining: 10.8ms
295:	learn: 0.3433151	total: 638ms	remaining: 8.62ms
296:	learn: 0.3431717	total: 640ms	remaining: 6.46ms
297:	learn: 0.3430367	total: 643ms	remaining: 4.31ms
298:	learn: 0.3428796	total: 645ms	remaining: 2.16ms
299:	learn: 0.3427289	total: 646ms	remaining: 0u

146:	learn: 0.3806940	total: 327ms	remaining: 340ms
147:	learn: 0.3805987	total: 330ms	remaining: 339ms
148:	learn: 0.3803674	total: 332ms	remaining: 336ms
149:	learn: 0.3800772	total: 334ms	remaining: 334ms
150:	learn: 0.3800301	total: 336ms	remaining: 331ms
151:	learn: 0.3797872	total: 337ms	remaining: 328ms
152:	learn: 0.3794055	total: 339ms	remaining: 326ms
153:	learn: 0.3791535	total: 341ms	remaining: 323ms
154:	learn: 0.3790413	total: 343ms	remaining: 321ms
155:	learn: 0.3789502	total: 345ms	remaining: 318ms
156:	learn: 0.3785811	total: 347ms	remaining: 316ms
157:	learn: 0.3783565	total: 349ms	remaining: 313ms
158:	learn: 0.3781427	total: 350ms	remaining: 310ms
159:	learn: 0.3777669	total: 352ms	remaining: 308ms
160:	learn: 0.3775417	total: 354ms	remaining: 305ms
161:	learn: 0.3772968	total: 356ms	remaining: 303ms
162:	learn: 0.3771069	total: 358ms	remaining: 301ms
163:	learn: 0.3769861	total: 360ms	remaining: 299ms
164:	learn: 0.3767324	total: 363ms	remaining: 297ms
165:	learn: 

14:	learn: 0.5032702	total: 32.1ms	remaining: 396ms
15:	learn: 0.4984265	total: 34.6ms	remaining: 398ms
16:	learn: 0.4949895	total: 36.4ms	remaining: 392ms
17:	learn: 0.4906008	total: 38.6ms	remaining: 390ms
18:	learn: 0.4853971	total: 40.9ms	remaining: 390ms
19:	learn: 0.4798768	total: 43.1ms	remaining: 388ms
20:	learn: 0.4757384	total: 45.3ms	remaining: 387ms
21:	learn: 0.4717122	total: 47.8ms	remaining: 387ms
22:	learn: 0.4691858	total: 49.9ms	remaining: 384ms
23:	learn: 0.4663976	total: 52.3ms	remaining: 384ms
24:	learn: 0.4623786	total: 54.5ms	remaining: 381ms
25:	learn: 0.4589188	total: 56.5ms	remaining: 378ms
26:	learn: 0.4560304	total: 58.4ms	remaining: 374ms
27:	learn: 0.4535642	total: 60.4ms	remaining: 371ms
28:	learn: 0.4518672	total: 62.6ms	remaining: 369ms
29:	learn: 0.4503303	total: 64.3ms	remaining: 364ms
30:	learn: 0.4479856	total: 65.8ms	remaining: 359ms
31:	learn: 0.4455558	total: 67.9ms	remaining: 356ms
32:	learn: 0.4434335	total: 70ms	remaining: 354ms
33:	learn: 0.4

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x0000027CFB200AD0>,
             param_grid={'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             verbose=3)

In [94]:
print("Best Parameters:",grid.best_params_)
print("Best Estimator:",grid.best_estimator_)

Best Parameters: {'max_depth': 3, 'n_estimators': 200}
Best Estimator: <catboost.core.CatBoostClassifier object at 0x0000027CF4182150>


In [95]:
cbc=CatBoostClassifier(max_depth=3,n_estimators=200)
cbc.fit(X_train,Y_train)
Y_pred=cbc.predict(X_test)
acc_cbc=round(dtc.score(X_train,Y_train)*100,2)
acc_cbc

Learning rate set to 0.042697
0:	learn: 0.6679394	total: 1.98ms	remaining: 395ms
1:	learn: 0.6497976	total: 4.03ms	remaining: 399ms
2:	learn: 0.6337569	total: 6.18ms	remaining: 406ms
3:	learn: 0.6163884	total: 8.08ms	remaining: 396ms
4:	learn: 0.5999898	total: 10.3ms	remaining: 401ms
5:	learn: 0.5884552	total: 12.6ms	remaining: 406ms
6:	learn: 0.5727810	total: 25ms	remaining: 688ms
7:	learn: 0.5644030	total: 26.5ms	remaining: 637ms
8:	learn: 0.5516424	total: 28.6ms	remaining: 607ms
9:	learn: 0.5408397	total: 30.2ms	remaining: 573ms
10:	learn: 0.5299379	total: 31.5ms	remaining: 541ms
11:	learn: 0.5217336	total: 33.1ms	remaining: 519ms
12:	learn: 0.5159854	total: 35.2ms	remaining: 507ms
13:	learn: 0.5079213	total: 37.3ms	remaining: 495ms
14:	learn: 0.5032702	total: 39.6ms	remaining: 488ms
15:	learn: 0.4984265	total: 41.4ms	remaining: 476ms
16:	learn: 0.4949895	total: 43.2ms	remaining: 465ms
17:	learn: 0.4906008	total: 44.6ms	remaining: 451ms
18:	learn: 0.4853971	total: 46.4ms	remaining: 

176:	learn: 0.3852630	total: 352ms	remaining: 45.7ms
177:	learn: 0.3850730	total: 353ms	remaining: 43.7ms
178:	learn: 0.3848461	total: 355ms	remaining: 41.7ms
179:	learn: 0.3847327	total: 357ms	remaining: 39.7ms
180:	learn: 0.3845983	total: 359ms	remaining: 37.6ms
181:	learn: 0.3844034	total: 361ms	remaining: 35.7ms
182:	learn: 0.3842842	total: 363ms	remaining: 33.7ms
183:	learn: 0.3841557	total: 366ms	remaining: 31.8ms
184:	learn: 0.3839841	total: 368ms	remaining: 29.8ms
185:	learn: 0.3837880	total: 370ms	remaining: 27.8ms
186:	learn: 0.3833675	total: 372ms	remaining: 25.9ms
187:	learn: 0.3829552	total: 374ms	remaining: 23.9ms
188:	learn: 0.3827007	total: 377ms	remaining: 22ms
189:	learn: 0.3824861	total: 379ms	remaining: 20ms
190:	learn: 0.3822020	total: 381ms	remaining: 18ms
191:	learn: 0.3820377	total: 383ms	remaining: 16ms
192:	learn: 0.3819839	total: 386ms	remaining: 14ms
193:	learn: 0.3818028	total: 388ms	remaining: 12ms
194:	learn: 0.3815678	total: 390ms	remaining: 9.99ms
195:	

86.04

In [96]:
cross_val_score(cbc,X_train,Y_train,scoring='accuracy',cv=10).mean()

Learning rate set to 0.040799
0:	learn: 0.6691075	total: 2ms	remaining: 399ms
1:	learn: 0.6513643	total: 4.04ms	remaining: 400ms
2:	learn: 0.6358835	total: 6.09ms	remaining: 400ms
3:	learn: 0.6188373	total: 8.21ms	remaining: 402ms
4:	learn: 0.6040714	total: 10.3ms	remaining: 400ms
5:	learn: 0.5894990	total: 12.4ms	remaining: 401ms
6:	learn: 0.5757370	total: 14.3ms	remaining: 395ms
7:	learn: 0.5671401	total: 16.4ms	remaining: 394ms
8:	learn: 0.5545949	total: 18.4ms	remaining: 391ms
9:	learn: 0.5444024	total: 20.2ms	remaining: 385ms
10:	learn: 0.5350646	total: 21.6ms	remaining: 372ms
11:	learn: 0.5270509	total: 23.6ms	remaining: 370ms
12:	learn: 0.5199903	total: 25.6ms	remaining: 369ms
13:	learn: 0.5134016	total: 27.1ms	remaining: 361ms
14:	learn: 0.5068753	total: 29ms	remaining: 357ms
15:	learn: 0.4998032	total: 29.9ms	remaining: 344ms
16:	learn: 0.4935391	total: 30.8ms	remaining: 332ms
17:	learn: 0.4888835	total: 32.9ms	remaining: 332ms
18:	learn: 0.4829277	total: 34.8ms	remaining: 332

175:	learn: 0.3876423	total: 332ms	remaining: 45.3ms
176:	learn: 0.3874377	total: 334ms	remaining: 43.4ms
177:	learn: 0.3871423	total: 336ms	remaining: 41.5ms
178:	learn: 0.3869982	total: 337ms	remaining: 39.6ms
179:	learn: 0.3866083	total: 339ms	remaining: 37.7ms
180:	learn: 0.3864773	total: 342ms	remaining: 35.9ms
181:	learn: 0.3863520	total: 344ms	remaining: 34ms
182:	learn: 0.3861414	total: 346ms	remaining: 32.2ms
183:	learn: 0.3859524	total: 349ms	remaining: 30.3ms
184:	learn: 0.3857147	total: 350ms	remaining: 28.3ms
185:	learn: 0.3854625	total: 352ms	remaining: 26.5ms
186:	learn: 0.3852633	total: 353ms	remaining: 24.5ms
187:	learn: 0.3850883	total: 355ms	remaining: 22.7ms
188:	learn: 0.3849230	total: 357ms	remaining: 20.8ms
189:	learn: 0.3846945	total: 359ms	remaining: 18.9ms
190:	learn: 0.3843844	total: 361ms	remaining: 17ms
191:	learn: 0.3842362	total: 363ms	remaining: 15.1ms
192:	learn: 0.3838786	total: 365ms	remaining: 13.2ms
193:	learn: 0.3837702	total: 367ms	remaining: 11.3

156:	learn: 0.3872837	total: 295ms	remaining: 80.8ms
157:	learn: 0.3869924	total: 297ms	remaining: 79ms
158:	learn: 0.3868979	total: 299ms	remaining: 77.1ms
159:	learn: 0.3868136	total: 300ms	remaining: 75.1ms
160:	learn: 0.3865964	total: 303ms	remaining: 73.4ms
161:	learn: 0.3863537	total: 305ms	remaining: 71.6ms
162:	learn: 0.3861930	total: 308ms	remaining: 69.8ms
163:	learn: 0.3860108	total: 310ms	remaining: 68ms
164:	learn: 0.3856670	total: 312ms	remaining: 66.1ms
165:	learn: 0.3853606	total: 314ms	remaining: 64.3ms
166:	learn: 0.3850725	total: 315ms	remaining: 62.3ms
167:	learn: 0.3849329	total: 317ms	remaining: 60.3ms
168:	learn: 0.3847785	total: 318ms	remaining: 58.4ms
169:	learn: 0.3842750	total: 321ms	remaining: 56.6ms
170:	learn: 0.3840315	total: 323ms	remaining: 54.7ms
171:	learn: 0.3838437	total: 325ms	remaining: 52.9ms
172:	learn: 0.3836294	total: 327ms	remaining: 51ms
173:	learn: 0.3834727	total: 329ms	remaining: 49.1ms
174:	learn: 0.3833285	total: 331ms	remaining: 47.2ms

141:	learn: 0.3775697	total: 273ms	remaining: 112ms
142:	learn: 0.3772501	total: 276ms	remaining: 110ms
143:	learn: 0.3769711	total: 278ms	remaining: 108ms
144:	learn: 0.3768363	total: 280ms	remaining: 106ms
145:	learn: 0.3766671	total: 282ms	remaining: 104ms
146:	learn: 0.3760102	total: 285ms	remaining: 103ms
147:	learn: 0.3758668	total: 287ms	remaining: 101ms
148:	learn: 0.3756655	total: 290ms	remaining: 99.1ms
149:	learn: 0.3754745	total: 292ms	remaining: 97.3ms
150:	learn: 0.3752916	total: 294ms	remaining: 95.3ms
151:	learn: 0.3752174	total: 296ms	remaining: 93.3ms
152:	learn: 0.3750982	total: 298ms	remaining: 91.4ms
153:	learn: 0.3748684	total: 300ms	remaining: 89.5ms
154:	learn: 0.3745711	total: 302ms	remaining: 87.6ms
155:	learn: 0.3741110	total: 303ms	remaining: 85.5ms
156:	learn: 0.3738124	total: 305ms	remaining: 83.6ms
157:	learn: 0.3736709	total: 307ms	remaining: 81.7ms
158:	learn: 0.3734788	total: 309ms	remaining: 79.6ms
159:	learn: 0.3730580	total: 311ms	remaining: 77.7ms


117:	learn: 0.4026293	total: 232ms	remaining: 161ms
118:	learn: 0.4024940	total: 233ms	remaining: 159ms
119:	learn: 0.4024163	total: 235ms	remaining: 157ms
120:	learn: 0.4022306	total: 237ms	remaining: 154ms
121:	learn: 0.4019788	total: 238ms	remaining: 152ms
122:	learn: 0.4019019	total: 240ms	remaining: 150ms
123:	learn: 0.4017449	total: 241ms	remaining: 148ms
124:	learn: 0.4014893	total: 242ms	remaining: 145ms
125:	learn: 0.4013066	total: 244ms	remaining: 143ms
126:	learn: 0.4012565	total: 246ms	remaining: 141ms
127:	learn: 0.4010477	total: 248ms	remaining: 139ms
128:	learn: 0.4008188	total: 249ms	remaining: 137ms
129:	learn: 0.4000686	total: 251ms	remaining: 135ms
130:	learn: 0.3999568	total: 252ms	remaining: 133ms
131:	learn: 0.3998812	total: 254ms	remaining: 131ms
132:	learn: 0.3993264	total: 256ms	remaining: 129ms
133:	learn: 0.3991085	total: 258ms	remaining: 127ms
134:	learn: 0.3987915	total: 260ms	remaining: 125ms
135:	learn: 0.3985207	total: 261ms	remaining: 123ms
136:	learn: 

...

Learning rate set to 0.040821
0:	learn: 0.6678662	total: 2.68ms	remaining: 534ms
1:	learn: 0.6525474	total: 5.13ms	remaining: 508ms
2:	learn: 0.6367837	total: 7.62ms	remaining: 500ms
3:	learn: 0.6195803	total: 10.5ms	remaining: 515ms
4:	learn: 0.6032562	total: 13.9ms	remaining: 540ms
5:	learn: 0.5859892	total: 15.9ms	remaining: 513ms
6:	learn: 0.5769473	total: 18.5ms	remaining: 510ms
7:	learn: 0.5622282	total: 21.4ms	remaining: 513ms
8:	learn: 0.5501108	total: 23.4ms	remaining: 497ms
9:	learn: 0.5424854	total: 24.5ms	remaining: 466ms
10:	learn: 0.5331062	total: 27.2ms	remaining: 467ms
11:	learn: 0.5236553	total: 29ms	remaining: 454ms
12:	learn: 0.5160623	total: 30.9ms	remaining: 444ms
13:	learn: 0.5083434	total: 32.9ms	remaining: 437ms
14:	learn: 0.5035991	total: 34.9ms	remaining: 430ms
15:	learn: 0.4993317	total: 37ms	remaining: 425ms
16:	learn: 0.4930755	total: 38.9ms	remaining: 419ms
17:	learn: 0.4881483	total: 41.1ms	remaining: 415ms
18:	learn: 0.4822146	total: 42.6ms	remaining: 40

Learning rate set to 0.040821
0:	learn: 0.6692497	total: 1.93ms	remaining: 385ms
1:	learn: 0.6525074	total: 3.84ms	remaining: 381ms
2:	learn: 0.6375908	total: 5.64ms	remaining: 370ms
3:	learn: 0.6213477	total: 6.87ms	remaining: 336ms
4:	learn: 0.6055155	total: 8.91ms	remaining: 348ms
5:	learn: 0.5945296	total: 10.4ms	remaining: 336ms
6:	learn: 0.5821887	total: 12.8ms	remaining: 353ms
7:	learn: 0.5735225	total: 14.9ms	remaining: 357ms
8:	learn: 0.5607146	total: 17.2ms	remaining: 364ms
9:	learn: 0.5508783	total: 18.5ms	remaining: 352ms
10:	learn: 0.5418690	total: 20ms	remaining: 343ms
11:	learn: 0.5339817	total: 22ms	remaining: 344ms
12:	learn: 0.5276265	total: 24.1ms	remaining: 346ms
13:	learn: 0.5219602	total: 26.2ms	remaining: 348ms
14:	learn: 0.5135240	total: 27.6ms	remaining: 340ms
15:	learn: 0.5061649	total: 28.9ms	remaining: 332ms
16:	learn: 0.5000354	total: 31.1ms	remaining: 334ms
17:	learn: 0.4952272	total: 33.2ms	remaining: 336ms
18:	learn: 0.4893769	total: 35.3ms	remaining: 33

Learning rate set to 0.040821
0:	learn: 0.6690656	total: 2.23ms	remaining: 444ms
1:	learn: 0.6513597	total: 4.48ms	remaining: 443ms
2:	learn: 0.6358107	total: 5.79ms	remaining: 381ms
3:	learn: 0.6193737	total: 8.01ms	remaining: 392ms
4:	learn: 0.6033166	total: 10.8ms	remaining: 420ms
5:	learn: 0.5880537	total: 13ms	remaining: 421ms
6:	learn: 0.5793658	total: 15.3ms	remaining: 422ms
7:	learn: 0.5664772	total: 17.4ms	remaining: 418ms
8:	learn: 0.5542170	total: 19.1ms	remaining: 405ms
9:	learn: 0.5466479	total: 21.4ms	remaining: 407ms
10:	learn: 0.5371528	total: 24.4ms	remaining: 419ms
11:	learn: 0.5303704	total: 26.8ms	remaining: 420ms
12:	learn: 0.5221922	total: 28.7ms	remaining: 413ms
13:	learn: 0.5138162	total: 30.5ms	remaining: 405ms
14:	learn: 0.5071841	total: 32.8ms	remaining: 405ms
15:	learn: 0.5003793	total: 35.4ms	remaining: 407ms
16:	learn: 0.4950252	total: 37.3ms	remaining: 402ms
17:	learn: 0.4909077	total: 40.3ms	remaining: 407ms
18:	learn: 0.4860159	total: 42.6ms	remaining: 

195:	learn: 0.3863795	total: 380ms	remaining: 7.75ms
196:	learn: 0.3862242	total: 381ms	remaining: 5.81ms
197:	learn: 0.3860124	total: 382ms	remaining: 3.86ms
198:	learn: 0.3857657	total: 384ms	remaining: 1.93ms
199:	learn: 0.3855415	total: 385ms	remaining: 0us


0.811567926455567

In [97]:
Y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [98]:
len(Y_pred)

418

In [99]:
ss.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [100]:
ss.shape

(418, 2)

In [101]:
final_prediction=pd.DataFrame({'Passenger ID': test.PassengerId, 'Survived': Y_pred})
final_prediction

,Passenger ID,Survived
881,892,0
882,893,0
883,894,0
884,895,0
885,896,0
...,...,...
1294,1305,0
1295,1306,1
1296,1307,0
1297,1308,0
